In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


import nltk
from nltk.stem import WordNetLemmatizer

from bs4 import BeautifulSoup
from wordcloud import WordCloud

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split

In [ ]:
"""
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')
"""

#### Chargement de données

In [ ]:
df = pd.read_csv('data\overflowdata.csv')

#### Déxouverte de données

In [ ]:
df.shape

In [ ]:
df.head(5)

In [ ]:
df.tail()

In [ ]:
df.describe()

In [ ]:
df.dtypes

In [ ]:
df.nunique()

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated(subset="Title").sum()

### Définition des fonctions de nettoyage

##### fonction de nettoyage des balises html

In [ ]:
# Définir une fonction pour nettoyer le html d'une chaîne
def clean_html(string):
# Créer un objet BeautifulSoup à partir de la chaîne
    soup = BeautifulSoup(string, "html.parser")
    # Extraire le texte de l'objet BeautifulSoup en ignorant les balises
    text = soup.get_text()
    return text

##### Fonction de lemmatization
(Tokenizer les phrases, nettoyage des stopwords et caractères spéciaux, lemmatisation)

In [ ]:
# Définir une fonction pour lemmatiser une phrase
def lemmatize_sentence(sentence):

    # Tokeniser la phrase en mots
    words = nltk.word_tokenize(sentence)
    stopwords = nltk.corpus.stopwords.words('english') + ['[', ']', ',', '.', ':', '?', '(', ')']

    words_w_stopwords = [i for i in words if i not in stopwords]

    # Lemmatiser chaque mot et reconstruire la phrase
    wnl = WordNetLemmatizer()
    lemmatized_words = [wnl.lemmatize(word) for word in words_w_stopwords]
    lemmatized_sentence = " ".join(lemmatized_words)

    return lemmatized_sentence

##### Application des fonctions de nettoyage

Nettoyage balises html de la colonne Body

In [ ]:
# Appliquer la fonction clean_html à la colonne Body du dataframe
df['Body_cleaned'] = df['Body'].apply(clean_html)

# Afficher le résultat
df['Body_cleaned']

Lemmatization de la colonne Body

In [ ]:
# Appliquer la fonction lemmatize_sentence à la colonne body du dataframe
df['Body_lemmatized'] = df['Body_cleaned'].apply(lemmatize_sentence)

# Afficher le résultat
df['Body_lemmatized']


Lemmatization de la colonne Title

In [ ]:
# Appliquer la fonction à la colonne title du dataframe
df['Title_lemmatized'] = df['Title'].apply(lemmatize_sentence)

# Afficher le résultat
df['Title_lemmatized']

Nettoyage de la colonne Tags des balises <>

In [ ]:
df['Tags'] = df['Tags'].str.replace('<', '').str.replace('>', ' ')

df['Tags'] = df['Tags'].str.split()

# Afficher le résultat
print(df['Tags'])

Définition d'un nouveau dataframe avec les deux colonnes Title et Body lemmatizé

In [ ]:
df.columns

In [ ]:
# afficher jusqu'à 100 caractères par colonne
pd.set_option('display.max_colwidth', 100)

# afficher tout le texte sans tronquer
pd.set_option('display.max_colwidth', None)

In [ ]:
df.loc[df['Id'] == 1250643,('Tags','Body', 'Body_cleaned', 'Body_lemmatized')]

In [ ]:
data = df.drop(['Title','Id', 'Score', 'ViewCount', 'FavoriteCount', 'AnswerCount', 'Body', 'Body_cleaned'], axis=1)

In [ ]:
data

In [ ]:
#Renommer les colonnes 
data = data.rename(columns={'Body_lemmatized': 'Body'})
data = data.rename(columns={'Title_lemmatized': 'Title'})

#### Représentation des mots de la colonne Title en WordCloud

In [ ]:
# Extraire la colonne des titres
titles = data['Title']

# Concaténer les titres en une seule chaîne de texte
text = " ".join(title for title in titles)

# Créer un objet WordCloud
wc = WordCloud(background_color="white", max_words=100)

# Générer le nuage de mots à partir du texte
wc.generate(text)

# Afficher le nuage de mots
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
# Extraire la colonne des titres
titles = data['Body']

# Concaténer les titres en une seule chaîne de texte
text = " ".join(title for title in titles)

# Créer un objet WordCloud
wc = WordCloud(background_color="white", max_words=100)

# Générer le nuage de mots à partir du texte
wc.generate(text)

# Afficher le nuage de mots
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
data = data.dropna()

In [ ]:
data.to_csv('data\dataframe.csv', encoding='utf-8', index=False, errors='ignore')

### Bag of words

1 - CountVectorizer

In [ ]:
# Créer une instance de CountVectorizer
vectorizer = CountVectorizer(lowercase=True,  ngram_range=(1,2), min_df=5, max_df=0.8)

df_train, df_test = train_test_split(data, test_size=0.2)

vectorizer.fit(df_train['Title'])

df_test['quest'] = df_test['Title'] + ' ' + df_test['Body']

# Appliquer le transformeur sur la colonne quest
counts_test_quest = vectorizer.transform(df_test['quest'])


vocab = vectorizer.get_feature_names()

counts = pd.DataFrame(counts_test_quest.toarray(), columns=vocab)



2 - TfidfVectorizer

In [ ]:
tf_vectorizer = TfidfVectorizer(lowercase=True,  ngram_range=(1,2), min_df=5, max_df=0.8)

df_train, df_test = train_test_split(data, test_size=0.2)

tf_vectorizer.fit(df_train['Title'])

# Créer une nouvelle colonne quest qui concatène Title et Body
df_test['quest'] = df_test['Title'] + ' ' + df_test['Body']

# Appliquer le transformeur sur la colonne quest
tdf_counts= tf_vectorizer.transform(df_test['quest'])

tf_vocab = tf_vectorizer.get_feature_names()


In [ ]:
# Afficher les scores tf-idf de chaque mot pour chaque document
for i, doc in enumerate(df_test['quest']):
    print(f"Document {i}: {doc}")

for j, word in enumerate(vocab):
    if tdf_counts[i,j] > 0:
        print(f"{word}: {tdf_counts[i,j]:.3f}")
    print()

In [ ]:
# afficher la taille du dataframe df_test
print(df_test.shape)

# afficher les premières lignes du dataframe df_test
print(df_test.head())

# afficher la taille de la matrice tdf_counts
print(tdf_counts.shape)

# afficher le nombre de valeurs non nulles dans la matrice tdf_counts
print(tdf_counts.nnz)